In [1]:
import argparse
import matplotlib as mlp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import trackpy as tp
from pathlib import Path
import argparse
import gzip
import pickle
import glob

In [2]:
input = '/data/groups/wheelenj/mosquitoes/20250106a01ard_20250106_111656.24568744'
    
hdf5_files = glob.glob(f"{input}/*.hdf5")
hdf5_files

['/data/groups/wheelenj/mosquitoes/20250106a01ard_20250106_111656.24568744/000001.hdf5',
 '/data/groups/wheelenj/mosquitoes/20250106a01ard_20250106_111656.24568744/000000.hdf5']

In [5]:
merged = merge_data(sorted(hdf5_files), input)


Getting data from 000000
5457 frames in 000000
Getting data from 000001
Skipping empty file: 000001
2946 rows in merged data frame.
Writing pickle file.


In [9]:
tracks = generate_tracks(merged, input)


Frame 80: 21 trajectories present.


KeyboardInterrupt: 

In [8]:
plot_tracks(tracks, args.input)



NameError: name 'plot_tracks' is not defined

In [4]:
def merge_data(hdf5s, input):

    all_data = []

    i = 0
    for file, i in zip(hdf5s, range(len(hdf5s))):
        with tp.PandasHDFStore(file, mode="r") as hdf5:
            print(f"Getting data from {Path(file).stem}")
            try:
                all_results = hdf5.dump()
                if i == 0:
                    records = int(all_results["frame"].max())
                    total_records = records
                    print(f"{records} frames in {Path(file).stem}")
                elif i != 0:
                    new_records = int(all_results["frame"].max())
                    total_records = total_records + new_records + 1
                    all_results["frame"] += total_records - new_records
                    print(
                        f"{new_records} frames in {Path(file).stem}. {total_records} total records")
                all_data.append(all_results)
            except ValueError as e:
                if "No objects to concatenate" in str(e):
                    print(f"Skipping empty file: {Path(file).stem}")
                    continue
                else:
                    raise e
            i += 1

    all_data = pd.concat(all_data)
    all_records = int(len(all_data["frame"]))
    print(f"{all_records} rows in merged data frame.")

    if "particle" in all_data.columns:
        all_data = all_data.drop(columns=["particle"])

    pickle_path = Path(input, Path(input).stem + "_tracks.pkl.gz")
    with gzip.open(pickle_path, "wb") as f:
        print("Writing pickle file.")
        pickle.dump(all_data, f)

    return all_data

In [7]:
def generate_tracks(df, input):

    if "miracidia" in input:
        search_range = 45
        memory = 25
        adaptive_stop = 30
    elif "mosquito" in input:
        search_range = 750
        memory = 100
        adaptive_stop = None
    elif "planaria" in input:
        search_range = 750
        memory = 100
        adaptive_stop = None

    print("Linking particles.")
    t = tp.link(df, search_range=search_range,
                memory=memory, adaptive_stop=adaptive_stop)
    pickle_path = Path(input, Path(input).stem + "_tracks.pkl.gz")
    with gzip.open(pickle_path, "wb") as f:
        print("Writing pickle file.")
        pickle.dump(t, f)
    print("Filtering stubs.")
    t1 = tp.filter_stubs(t, 200)

    return t1

In [ ]:
def plot_tracks(tracks, input):

    print("Plotting trajectories.")
    save_path = Path(input, Path(input).stem + ".pdf")
    fig = plt.figure()
    ax = plt.gca()
    tp.plot_traj(tracks, ax=ax)
    fig.savefig(save_path)

In [11]:
str = "20250106a01ard_20250106_111656.24568709"
result = str.rsplit('.', 1)[0]
result

'20250106a01ard_20250106_111656'